In [5]:
"""(0) Problem 
* 입력 데이터 : size_bytes: 크기(바이트) / price: 가격 금액 / rating_count_tot: 사용자 등급 수 (모든 버전) / 
  rating_count_ver: 사용자 등급 수 (현재 버전) / user_rating_ver: 평균 사용자 등급 값 (현재 버전) / ver: 최신 버전 코드 / 
  cont_rating: 콘텐츠 등급 / prime_genre: 기본 장르 / ipadSc_urls.num: 표시 할 스크린 샷 수 / lang.num: 지원되는 언어 수

* 출력 데이터 : user_rating: 평균 사용자 등급 값 (모든 버전)

* 문제 유형 : 회귀

* 레이블 칼럼 갯수 : 7197 rows × 17 columns(전처리 전), 7189 rows × 11 columns(전처리 후)
"""

from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd


"""(3) Measure 
* 평균 절대 오차(Mean absolute error) 메소드 정의
* 회귀분석으로 나온 예측값과 실제 값을 빼서 나온 값의 절대값을 평균냄
* MAE는 낮을수록 좋음
"""
def MAE(test_y, pred_y):
    return np.mean(np.abs((test_y - pred_y)))

# csv 파일 불러오기
file_path = '/home/u1043/machinelearning/practice/datasets_30069_39285_AppleStore.csv'
store_data = pd.read_csv(file_path)

# 데이터프레임 정보 출력
# store_data.info()


"""(1) Feature 

* index, id, track_name(어플 이름)은 각 어플마다의 고유값이기 때문에 drop

* sub_devices.num(지원 장치 수), 통화 종류(currency), vpp_lic(vpp 장치 기반 라이센스 활성화) feature는 
  모든 어플에 대해 같거나 크게 차이가 없어서 drop

* ver(최신 버전 코드)는 .으로 숫자가 연결(2.7.4 이런식으로) 되어있어서 첫번째 자리만 잘라서 ver 열의 값으로 저장

* ver 값이 문자열로 되어있는 행은 삭제

* 문자열 값을 입력 값으로 허락하지 않기 때문에 
  prime_genre(기본 장르)와 cont_rating(콘텐츠 등급)와 같은 범주형 데이터는 숫자형으로 레이블인코딩

"""
# final_data2.isnull().sum() # 결측 데이터가 있나 확인, 결측 데이터 없음

# 데이터 전처리 
# 필요 없는 feature Drop
del(store_data['Unnamed: 0']) # index 행
del(store_data['id']) # id값
del(store_data['track_name']) # 어플 이름
del(store_data['sup_devices.num']) # 지원 장치 수
del(store_data['currency']) # 통화 종류
del(store_data['vpp_lic']) # Vpp 장치 기반 라이센스 활성화 

# ver열의 데이터들의 .을 기준으로 첫번째 숫자만 저장하기
ver_split = store_data["ver"].str.split(".")
store_data["ver"] = ver_split.str.get(0)

# ver 열의 값들 중 문자열이 있는 행은 삭제
contains_string_df = store_data['ver'].str.contains('iOV 2|v2|v3|1,2|V3|v1|Update 1')
store_data = store_data[~contains_string_df]

# 범주형 데이터는 레이블인코딩
prime_genre = preprocessing.LabelEncoder()
final_data1 = store_data.copy()
final_data1['prime_genre'] = prime_genre.fit_transform(final_data1['prime_genre'])

cont_rating = preprocessing.LabelEncoder()
final_data2 = final_data1.copy()
final_data2['cont_rating'] = prime_genre.fit_transform(final_data2['cont_rating'])

# 전처리 후 데이터프레임 정보 출력
#final_data2.info()

# 사용할 feature들 변수에 담아주기
features = ['size_bytes', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'ipadSc_urls.num', 'lang.num']

"""(3) Measure
* K- fold 교차 검증 사용
* 10개의 fold를 만들어서 교차검증 진행
* 데이터 셋에 대하여 정확도를 향상시키기 위해 사용
"""
kf = KFold(n_splits=10, shuffle=True) # 데이터를 10개로 나누기, suffle 사용하여 데이터를 무작위로 섞어줌

accrs = []

fold_idx = 1

for train_idx, test_idx in kf.split(final_data2): # 10번 반복
    train_d, test_d = final_data2.iloc[train_idx], final_data2.iloc[test_idx] # 학습 데이터, 훈련 데이터 나누기
    
    """(4) Model parameter engineering 
    * 데이터 스케일링 
    * size_bytes: 크기(바이트)와 같은 변수들의 단위 차이로 인해 숫자의 스케일이 크게 달라지는 경우가 있어서 
    * 모든 feature 값이 0~1 사이에 있도록 MinMaxScaler 사용
    * Scaler를 사용하기 전보다 MSE 값이 감소
    
    * PCA를 사용하였을 때 MSE가 더 높게 올라가서 사용하지 않음
    """
    scaler = MinMaxScaler()
    train_d_scale = scaler.fit_transform(train_d[features])
    test_d_scale = scaler.transform(test_d[features])
    
    # PCA를 사용하였을 때 MSE가 더 올라감 
    #pca = PCA(n_components=10)

    train_y = train_d['user_rating']
    train_x = train_d_scale

    test_y = test_d['user_rating']
    test_x = test_d_scale

    """(2)Model
    * 모델 : LinearRegression 
    * Linear Regression, Lasso Regression, Ridge Regression 세 모델을 학습시킨 결과 
    * Linear Regression이 다른 두 모델에 비해 더 낮은 MAE가 나왔기 때문에 LinearRegression 선택
    """
    model = LinearRegression()
    model.fit(train_x, train_y)
    
    # test_x에 대한 예측값 pred_y 구하기
    pred_y = model.predict(test_x)
    
    #모델 학습 후, 실제 값과 예측 값을 파라메터로 전달하여 MAE 메소드 호출
    mae = MAE(test_y, pred_y)
    
    accrs.append(mae)
    
    print('fold {}'.format(fold_idx), ": MAE =", mae, "\n")
    
    fold_idx += 1

print("Total (Average) MAE(Mean absolute error) = ", np.average(accrs)) # 총 10번의 fold 평균 구하기

fold 1 : MAE = 0.619818255038538 

fold 2 : MAE = 0.6165081823487862 

fold 3 : MAE = 0.613108307187064 

fold 4 : MAE = 0.648640391068741 

fold 5 : MAE = 0.6189114106934778 

fold 6 : MAE = 0.6183159614994924 

fold 7 : MAE = 0.6426403206265185 

fold 8 : MAE = 0.655577851489653 

fold 9 : MAE = 0.643987821984524 

fold 10 : MAE = 0.6656324295418047 

Total (Average) MAE(Mean absolute error) =  0.6343140931478599
